In [ ]:
import json
import folium
from folium.plugins import HeatMap
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import io
import os
import imageio
from nextbike_functions import *

In [3]:
def two_hours_forward(timestamp_string: str):
    date, time = timestamp_string.split(' ')
    year, month, day = date.split('-')
    hour, minute, second = time.split(':')
    date = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
    date += datetime.timedelta(hours=2)
    return date

In [4]:
def five_minutes_back(timestamp_string: str):
    date, time = timestamp_string.split(' ')
    year, month, day = date.split('-')
    hour, minute, second = time.split('.')
    date = datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second))
    date -= datetime.timedelta(minutes=5)
    return date

In [5]:
data = []
timestamp_dictionary = dict()
filenames = []

i = 0
# Extracting stations info for stations in Warsaw
with open('json_files/2024-06-29-veturilo.json', 'r', encoding='utf8') as file:
    for line in file:
        data_piece = json.loads(line)
        if data_piece['city_uid'] == 812:
            timestamp = str(two_hours_forward(data_piece['timestamp']['$date'].replace('T', ' ')[:-1])).replace(':', '.')
            if not timestamp in timestamp_dictionary.keys():
                timestamp_dictionary[timestamp] = []
            timestamp_dictionary[timestamp].append(data_piece)

In [6]:
freestanding_coordinates_data = {}
abandonment_data = {}
index = 0

for timestamp, data in timestamp_dictionary.items():
    freestanding_coordinates_data[timestamp] = []
    abandonment_data[timestamp] = []
    for station in data:
        if station['bike']:
            station['shortened_lat'] = round(float(station['lat']), 3)
            station['shortened_lng'] = round(float(station['lng']), 3)
            if index == 0:
                abandonment_data[timestamp].append((station['bike_numbers'][0], station['lat'], station['lng']))
            else:
                abandonment_data[timestamp].append((station['bike_numbers'][0], station['lat'], station['lng']))
            freestanding_coordinates_data[timestamp].append((station['lat'], station['lng']))
    if index != 0:
        abandonment_data[timestamp].extend(abandonment_data[list(abandonment_data.keys())[index - 1]])
    index += 1
    
for key in abandonment_data.keys():
    abandonment_data[key] = list(set(abandonment_data[key]))

In [7]:
non_disappearing_data = {}
for key, value in abandonment_data.items():
    non_disappearing_data[key] = []
    for bike_number, lat, lng in value:
        non_disappearing_data[key].append((lat, lng))

In [1]:
used_font = ImageFont.truetype('arial.ttf', 30)
target_folder = 'heatmaps/'

if not os.path.exists(target_folder):
    os.mkdir(target_folder)

for timestamp, freestanding_coordinates in non_disappearing_data.items():
    m = folium.Map(location=[52.2265, 21.0196], zoom_start=14)
    HeatMap(freestanding_coordinates).add_to(m)
    filename = target_folder + timestamp + '.png'
    img_data = m._to_png(0)
    img = Image.open(io.BytesIO(img_data))
    
    I1 = ImageDraw.Draw(img)
    I1.text((img.size[0]-350, img.size[1]-80), timestamp, font = used_font, fill = (0, 0, 0))
    
    img.save(filename)
    filenames.append(filename)
    print(os.path.basename(filename), "done")

images = []
for filename in filenames:
    images.append(imageio.v2.imread(filename))
imageio.mimsave('movie.gif', images)

NameError: name 'ImageFont' is not defined